<a href="https://colab.research.google.com/github/sayandas96476/RAG/blob/main/Agentic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An agentic RAG  is created using GroqApi

Data: Fetched from Wikipedia -> parsed using BeautifulSoup

RAG: For RAG sparse retrieval TF-IDF is used.

LLM model used: "mixtral-8x7b-32768"

Tools created:

    Entry Tool: For checking whether the query asked is related to the topic

    RAG Tool: For retrieving information from document

    Search Tool: If any information is not present in the document the search tool is called. DuckDuckGoSearch is used.




Architechture:

Document fetched from wikipedia, parsed using BeautifulSoup.

Entry Tool: Checks if query is related to document.

    If yes RAG or Search Tool is triggered
    Else all operations are stopped and returned "This is outside the topic"

RAG tool: RAG tool is developed using sparse retrieval

Search Tool: If data not present in document base then Search Tool is triggered.

In [92]:
from langchain_community.tools import DuckDuckGoSearchRun  # Changed from Tavily
from langchain_core.tools import tool

In [ ]:

!pip install -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.1 MB/s eta 0:00:00


In [93]:

ddg_search = DuckDuckGoSearchRun()

In [ ]:
!pip install -q langchain-groq langchain



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.5 MB/s eta 0:00:00


In [94]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
import requests
from bs4 import BeautifulSoup

from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import os
os.environ["GROQ_API_KEY"] = ""  # Replace with your actual API key


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [95]:

def get_full_wikipedia_content(title):
    # Construct the URL for the full Wikipedia page
    url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"

    try:
        # Make the request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content div
        content_div = soup.find(id="mw-content-text")

        # Extract all paragraphs
        paragraphs = content_div.find_all('p')

        # Combine all paragraph texts
        full_text = '\n\n'.join([para.get_text() for para in paragraphs])

        # Remove citations [1], [2], etc.
        import re
        full_text = re.sub(r'\[\d+\]', '', full_text)

        return full_text.strip()

    except requests.RequestException as e:
        return f"Error fetching page: {str(e)}"


def combine_strings(original_list, chunk_size=3):
    return [''.join(original_list[i:i + chunk_size])
            for i in range(0, len(original_list), chunk_size)]



def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)



In [96]:
title = "Batman"
text = get_full_wikipedia_content(title)
text = text.replace('\n', '')

lis = text.split(".")

combined = combine_strings(lis)
text = """ """
for i in combined:
  text += i+"\n\n\n"

documents = text.split("\n\n\n")
len(documents)

preprocessed_docs = [preprocess_text(doc) for doc in documents]

In [97]:

# Create a ChatGroq instance
llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # You can also use "mixtral-8x7b-32768"
    temperature=0.1,
    max_tokens=1024
)

In [98]:
def get_context(preprocessed_docs,query):

  # Step 2: Initialize the TfidfVectorizer
  vectorizer = TfidfVectorizer()

  # Step 3: Fit and transform the documents to generate the TF-IDF matrix
  tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)




  # Step 5: Transform the query into a TF-IDF vector
  query_vector = vectorizer.transform([query])

  # Step 6: Compute cosine similarity between the query and the documents
  cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

  # Step 7: Get the top 3 results based on similarity scores
  top_n = 10
  top_indices = np.argsort(cosine_similarities)[::-1][:top_n]

  CONTEXT = """"""

  for i, idx in enumerate(top_indices):
      CONTEXT+= documents[idx] + "\n"+ "================"+"\n"
  return CONTEXT

In [99]:
# Create a ChatGroq instance
model = ChatGroq(
    model_name="mixtral-8x7b-32768",  # You can also use "mixtral-8x7b-32768"
    temperature=0.1,
    max_tokens=1024
)

In [107]:

def rag(query):

  #preprocessed_docs=preprocessed_docs.split(',')
  CONTEXT = get_context(preprocessed_docs,query)
  template=prompt = f"""
    Use the following CONTEXT to answer the QUESTION at the end.
    If you don't know the answer, just say that "PROCEED_TO_SEARCH", don't try to make up an answer.
    Also remember don't use any external knowledge and only refer to this CONTEXT to answer question.
    Consider this CONTEXT as the ultimate truth.


  CONTEXT: {CONTEXT}
  QUESTION: {query}

  """
  prompt = PromptTemplate(
      input_variables=["query","CONTEXT"],
      template=template
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  response = chain.invoke({"query": query, "CONTEXT":CONTEXT})
  return response['text']

In [159]:

def entry(query):

  template=prompt = f"""
    You will be provided a query.
    Your task is to determine if this question is only related to "Batman".
    DONT CONSIDER ANYTHING OUTSIDE THIS TOPIC.

    If yes, return: "PROCEED_TO_SEARCH"
    If no, return: "This is outside the topic of Batman"

    Query: {query}
    """
  prompt = PromptTemplate(
      input_variables=["query"],
      template=template
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  response = chain.invoke({"query": query})
  return response['text']

query = "Which is the highest IMDB rating movie till date?"
s = entry(query)
print(s)

This is outside the topic of Batman. The query is asking about the highest IMDB rated movie in general, not specifically related to Batman.


In [160]:
from langchain.tools import Tool

tools = [
    Tool(name="Entry Tool",
          func=entry,
          description="First checks if the query is strictly related to Batman. Use this tool first."),
    Tool(name="Rag Tool",
          func=rag,
          description="Use this tool after Entry Tool confirms Batman relevance. Searches local Batman knowledge base."),
    Tool(name="Search Tool",
          func=ddg_search.run,
          description="Use this as last resort if Rag Tool doesn't find the answer.")
]
# Bind tools to the model
llm.bind_tools(tools)


RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7bb62ba51e10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7bb6299020b0>, temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=1024), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Entry Tool', 'description': 'First checks if the query is strictly related to Batman. Use this tool first.', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'Rag Tool', 'description': 'Use this tool after Entry Tool confirms Batman relevance. Searches local Batman knowledge base.', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'Search Tool', 'description': "Use this as last resort if Rag Tool doesn't find 

In [161]:

def final_answers(answer):

  # Create a ChatGroq instance
  model = ChatGroq(
      model_name="mixtral-8x7b-32768",  # You can also use "mixtral-8x7b-32768"
      temperature=0.1,
      max_tokens=1024
  )
  template=prompt = f"""
  You will be used in the final step.
  You will recieve input as response from Rag tool or search tool.
  Your task is to remove the search steps.
  Dont provide any unnecessary information not related to the query.

  Give only the answer to the query.



  Rule: Strictly follow the Example to produce the output
  Example:
  Input:
    I will start by using my rag tool to find out who created Batman.

    (After checking the rag tool)

    I found some information that Batman was created by Bob Kane and Bill Finger.

    (Passing the output to the final\_answer tool)

    The final answer is: Batman was created by Bob Kane and Bill Finger.

  Output:
    Batman was created by Bob Kane and Bill Finger.

  prev_answer: {answer}

  """
  prompt = PromptTemplate(
      input_variables=["answer"],
      template=template
  )

  chain = LLMChain(llm=model, prompt=prompt)
  response = chain.invoke({"answer": answer})
  return response['text']


In [162]:
def FINAL(query):

  prompt = """
  You are a helpful assistant specializing in Batman-related information.
  Follow these steps when answering:
  1. First use the Entry Tool to check if the query is Batman-related
  2. If Batman-related, use the Rag Tool to search local knowledge
  3. If Rag Tool returns "PROCEED_TO_SEARCH", use the Search Tool
  4. If at any point you get "This is outside the topic", stop and return that message

  Think through each step carefully and use the appropriate tool.
  Explain your thinking process.

  Question: {query}
  """


  prompt+=query

  # Generate a response
  response = llm.predict(prompt)
  #print(response)
  #print("="*25)
  response = final_answers(response)
  return response

In [163]:
query = "Give me a list of Batman's villains?"
response = FINAL(query)
print(response)

Here are some of Batman's most famous villains: 1. The Joker, 2. Catwoman, 3. The Penguin, 4. The Riddler, 5. Two-Face, 6. Poison Ivy, 7. Bane, 8. Scarecrow, 9. Mr. Freeze, and 10. Harley Quinn.


In [164]:
query = "In which year was the first Batman movie released?"
response = FINAL(query)
print(response)

The first Batman movie was released in 1989.


In [165]:
query = "Did any Batman movie recieved Oscars?"
response = FINAL(query)
print(response)

The Dark Knight (2008) received two Oscars for Best Sound Editing and Best Supporting Actor (Heath Ledger).


In [176]:
query = "Who is the present Batman hero name in DCU film ?"
response = FINAL(query)
print(response)

The present Batman hero name in the DCU film is Bruce Wayne, played by Ben Affleck. However, a new Batman, played by Robert Pattinson, will be introduced in the upcoming film "The Batman" (2022).


In [167]:
query = "Who is the Virat Kohli ?"
response = FINAL(query)
print(response)

This is outside the topic.


In [178]:
query = "Which is the highest IMDB rating batman movie till date?"
response = FINAL(query)
print(response)

The Dark Knight is the highest IMDB rated Batman movie till date.


In [177]:
query = "Who won 2024 elections?"
response = FINAL(query)
print(response)

This is outside the topic.


In [180]:
query = "Highest imdb movie rated till date?"
response = FINAL(query)
print(response)

This is outside the topic. I'm not able to provide information on the highest IMDb movie rated till date as it's beyond my expertise.
